In [1]:
import os
import zipfile
import pandas as pd
import shutil

def get_gtfs_zip_files(directory):
    """
    指定されたディレクトリ内のZIPファイルを検索する関数
    
    Args:
    directory (str): ZIPファイルが格納されているディレクトリ
    
    Returns:
    list: 見つかったZIPファイルのパスリスト
    """
    zip_files = []
    
    # ディレクトリ内のファイルをすべて走査
    for filename in os.listdir(directory):
        # .zipで終わるファイルを選択
        if filename.lower().endswith('.zip'):
            full_path = os.path.join(directory, filename)
            zip_files.append(full_path)
    
    # ファイルパスをアルファベット順にソート
    zip_files.sort()
    
    return zip_files

def extract_gtfs_from_zip(zip_path, extract_dir):
    """
    ZIPファイルを指定されたディレクトリに展開する関数
    
    Args:
    zip_path (str): ZIPファイルパス
    extract_dir (str): 展開先ディレクトリ
    
    Returns:
    str: 展開されたディレクトリのパス
    """
    os.makedirs(extract_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    return extract_dir

def get_agency_name(extract_dir):
    """
    展開されたディレクトリからagency_nameを取得する関数
    
    Args:
    extract_dir (str): 展開されたディレクトリのパス
    
    Returns:
    str: agency_name
    """
    agency_path = os.path.join(extract_dir, 'agency.txt')
    try:
        if os.path.exists(agency_path):
            agency_df = pd.read_csv(agency_path)
            if 'agency_name' in agency_df.columns and not agency_df.empty:
                return agency_df['agency_name'].iloc[0]
    except Exception as e:
        print(f"Warning: {agency_path} からagency_nameの取得に失敗しました: {e}")
    
    # 取得できない場合はディレクトリ名を返す
    return os.path.basename(extract_dir)

def merge_gtfs_with_template(template_zip_path, gtfs_directory, output_directory, output_directory_gtfs):
    """
    テンプレートを基準に複数のGTFSフィードをマージする関数
    
    Args:
    template_zip_path (str): テンプレートのZIPファイルパス
    gtfs_directory (str): マージするデータのZIPファイルが格納されたディレクトリ
    output_directory (str): マージ後のデータを保存するディレクトリ
    """
    # 出力ディレクトリ作成
    os.makedirs(output_directory_gtfs, exist_ok=True)
    
    # ディレクトリ内のZIPファイルを検出
    gtfs_zip_paths = get_gtfs_zip_files(gtfs_directory)
    print(f"検出されたZIPファイル: {gtfs_zip_paths}")
    
    # 一時ディレクトリ作成
    temp_dir = os.path.join(output_directory_gtfs, 'temp')
    os.makedirs(temp_dir, exist_ok=True)
    
    # calendar.txtの曜日カラムリストとtrips.txtの除外カラム
    calendar_day_columns = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    trips_exclude_columns = ['direction_id']
    
    try:
        # テンプレートZIPを展開
        template_extract_dir = extract_gtfs_from_zip(template_zip_path, os.path.join(temp_dir, 'template'))
        
        # テンプレートファイルのリストを取得
        template_files = [f for f in os.listdir(template_extract_dir) 
                         if f.endswith('.txt') and f != 'fare_rules.txt']
        print(f"テンプレートファイル: {template_files}")
        
        # プロバイダー対応表のデータを保持するリスト
        mapping_data = []
        
        # ファイルごとのマージデータを保持する辞書
        merged_data = {}
        for filename in template_files:
            if filename == 'trips.txt':
                # trips.txtの場合、direction_idを整数型として読み込む
                merged_data[filename] = [pd.read_csv(os.path.join(template_extract_dir, filename), dtype={'direction_id': 'Int64'})]
            elif filename == 'routes.txt':
                # routes.txtの場合、route_text_colorを文字列として読み込む
                merged_data[filename] = [pd.read_csv(os.path.join(template_extract_dir, filename), dtype={'route_text_color': str})]
            else:
                merged_data[filename] = [pd.read_csv(os.path.join(template_extract_dir, filename))]
        
        # 各GTFSデータを処理
        for i, zip_path in enumerate(gtfs_zip_paths, 1):
            # ZIPを展開
            provider_dir = extract_gtfs_from_zip(zip_path, os.path.join(temp_dir, f'provider_{i}'))
            
            # agency_nameを取得
            agency_name = get_agency_name(provider_dir)
            
            # プロバイダー対応表に追加
            mapping_data.append({
                'provider_id': f'provider_{i}',
                'zip_filename': os.path.basename(zip_path),
                'agency_name': agency_name
            })
            
            # 各テンプレートファイルについて処理
            for filename in template_files:
                file_path = os.path.join(provider_dir, filename)
                
                if os.path.exists(file_path):
                    # ファイルの種類に応じて適切なデータ型で読み込む
                    if filename == 'trips.txt':
                        df = pd.read_csv(file_path, dtype={'direction_id': 'Int64'})
                    elif filename == 'routes.txt':
                        df = pd.read_csv(file_path, dtype={'route_text_color': str})
                    else:
                        df = pd.read_csv(file_path)
                    
                    # テンプレートと同じカラムに揃える
                    df = df.reindex(columns=merged_data[filename][0].columns)
                    
                    # IDカラムがある場合は接頭辞を追加（calendar.txtの曜日カラムとtrips.txtの特定カラムを除外）
                    id_columns = [col for col in df.columns 
                                if 'id' in col.lower() and 
                                (filename != 'calendar.txt' or col not in calendar_day_columns) and
                                (filename != 'trips.txt' or col not in trips_exclude_columns)]
                    
                    for col in id_columns:
                        if col in df.columns:
                            if filename == 'trips.txt' and (col == 'block_id' or col == 'shape_id'):
                                # block_idとshape_idの場合、空でない値にのみ接頭辞を追加
                                df[col] = df[col].apply(lambda x: f'provider_{i}_{x}' if pd.notna(x) and str(x).strip() != '' else x)
                            elif filename == 'translations.txt' and (col == 'record_sub_id' or col == 'record_id'):
                                # record_idとrecord_sub_idの場合、空でない値にのみ接頭辞を追加
                                df[col] = df[col].apply(lambda x: f'provider_{i}_{x}' if pd.notna(x) and str(x).strip() != '' else x)
                            else:
                                df[col] = f'provider_{i}_' + df[col].astype(str)
                    
                    # route_text_colorの特別処理
                    if filename == 'routes.txt' and 'route_text_color' in df.columns:
                        df['route_text_color'] = df['route_text_color'].apply(lambda x: x if pd.isna(x) or str(x).strip() == '' else str(x))
                    
                    merged_data[filename].append(df)
        
        # プロバイダー対応表を出力
        provider_mapping = pd.DataFrame(mapping_data)
        mapping_path = os.path.join(output_directory, 'provider_mapping.txt')
        provider_mapping.to_csv(mapping_path, index=False)
        print(f"プロバイダー対応表を {mapping_path} に出力しました")
        
        # 各ファイルのマージデータを出力
        for filename, dataframes in merged_data.items():
            final_merged_df = pd.concat(dataframes, ignore_index=True)
            
            # trips.txtの場合、direction_idが整数値として出力されるようにする
            if filename == 'trips.txt' and 'direction_id' in final_merged_df.columns:
                final_merged_df['direction_id'] = final_merged_df['direction_id'].astype('Int64')
            
            # routes.txtの場合、route_text_colorが文字列として出力され、空欄は空欄のまま保持する
            if filename == 'routes.txt' and 'route_text_color' in final_merged_df.columns:
                final_merged_df['route_text_color'] = final_merged_df['route_text_color'].apply(
                    lambda x: x if pd.isna(x) or str(x).strip() == '' else str(x))
            
            # translations.txtの場合、指定された5カラムで重複チェックを行い、最初の行のみを残す
            if filename == 'translations.txt':
                final_merged_df = final_merged_df.drop_duplicates(
                    subset=['table_name', 'field_name', 'language', 'field_value'],
                    keep='first'
                )
            
            output_path = os.path.join(output_directory_gtfs, filename)
            final_merged_df.to_csv(output_path, index=False)
            print(f"{filename} をマージしました")
        
        print("フィードのマージが完了しました")
    
    finally:
        # 一時ディレクトリを削除
        shutil.rmtree(temp_dir, ignore_errors=True)

In [2]:

def main():
    # テンプレートZIPファイルのパス
    template_zip_path = 'GTFS_template/GTFS_template.zip'
    
    # データのZIPファイルが格納されているディレクトリ
    gtfs_directory = 'input/gtfses_kanazawa'
    
    # 統合されたデータを保存するディレクトリ
    output_directory = 'output'
    output_directory_gtfs = os.path.join(output_directory, 'merged_gtfs')
    
    # マージ関数を呼び出し
    merge_gtfs_with_template(template_zip_path, gtfs_directory, output_directory, output_directory_gtfs)
    
    # ZIP化を実行
    filename = os.path.join(output_directory, 'merged_gtfs')
    shutil.make_archive(filename,'zip',output_directory_gtfs)

if __name__ == "__main__":
    main()

検出されたZIPファイル: ['input/gtfses_kanazawa\\20240401flatbus_rev.zip', 'input/gtfses_kanazawa\\feed_kaetsunou_kaetsunouippan_20241201_20241106101715.zip', 'input/gtfses_kanazawa\\gtfs-hokuriku-railway-20240316_rev.zip', 'input/gtfses_kanazawa\\gtfs-hokutetsu2024.zip', 'input/gtfses_kanazawa\\hokutetsu.zip', 'input/gtfses_kanazawa\\machibus.zip', 'input/gtfses_kanazawa\\nishinihonjrbus.zip', 'input/gtfses_kanazawa\\notty_current.zip']
テンプレートファイル: ['agency.txt', 'calendar.txt', 'calendar_dates.txt', 'feed_info.txt', 'frequencies.txt', 'routes.txt', 'shapes.txt', 'stops.txt', 'stop_times.txt', 'transfers.txt', 'translations.txt', 'trips.txt']
プロバイダー対応表を output\provider_mapping.txt に出力しました
agency.txt をマージしました
calendar.txt をマージしました
calendar_dates.txt をマージしました
feed_info.txt をマージしました
frequencies.txt をマージしました
routes.txt をマージしました


C:\Users\tachibana\AppData\Local\Temp\ipykernel_31472\975870671.py:178: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_merged_df = pd.concat(dataframes, ignore_index=True)


shapes.txt をマージしました
stops.txt をマージしました
stop_times.txt をマージしました
transfers.txt をマージしました
translations.txt をマージしました
trips.txt をマージしました
フィードのマージが完了しました
